In [1]:
import pandas as pd
from itertools import chain
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt

In [2]:
states = pd.read_csv('states.csv')
cities = pd.read_csv('cities.csv')
data = pd.read_csv('FoMCData.csv', error_bad_lines=False)

In [3]:
data
product_ID = data['Item'].str.replace('Product','')
data['product_ID'] = product_ID
data.to_csv('final.csv')
data.columns


Index(['Nos.', 'order_id', 'Item', 'Price', 'City', 'Quantity', 'Bill Amount',
       'State of Delivery', 'Year', 'product_ID'],
      dtype='object')

In [25]:
data['product_ID'] = data['product_ID'].str.strip()
data.dropna(axis=0, subset=['order_id'], inplace=True)

data['order_id'] = data['order_id'].astype('str')
#df = df[~df['InvoiceNo'].str.contains('C')]
data.head(10)
#data['']

,Nos.,order_id,Item,Price,City,Quantity,Bill Amount,State of Delivery,Year,product_ID
0,1,1,Product 42,83,Pune,4,332,Maharashtra,2014,42
1,2,1,Product 462,58,Pune,6,348,Maharashtra,2014,462
2,3,1,Product 305,277,Pune,6,1662,Maharashtra,2014,305
3,4,1,Product 388,242,Pune,7,1694,Maharashtra,2014,388
4,5,1,Product 128,261,Pune,7,1827,Maharashtra,2014,128
5,6,1,Product 163,124,Pune,2,248,Maharashtra,2014,163
6,7,1,Product 217,68,Pune,9,612,Maharashtra,2014,217
7,8,1,Product 32,192,Pune,1,192,Maharashtra,2014,32
8,9,1,Product 187,103,Pune,10,1030,Maharashtra,2014,187
9,10,1,Product 236,65,Pune,9,585,Maharashtra,2014,236


In [26]:
basket = (data.groupby(['order_id', 'product_ID'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))
basket.shape

(500, 500)

In [28]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head(4)

product_ID,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
frequent_itemsets = apriori(basket_sets, min_support=0.013, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(10)
#len(rules)

,antecedants,consequents,support,confidence,lift
0,(236),(206),0.116,0.120690,2.873563
1,(206),(236),0.042,0.333333,2.873563
2,(173),(93),0.112,0.125000,1.136364
3,(93),(173),0.110,0.127273,1.136364
4,(173),(250),0.112,0.125000,4.464286
5,(250),(173),0.028,0.500000,4.464286


In [39]:
def flatten_list(templist):
	return list(chain.from_iterable(templist))
def locator(x):
    a = []
    z = []
    cities = states['City'].str.lower()
    try:
        for i in x:
            x1 = states.loc[cities == i.lower()]
            z.append(list(set(x1['State'].tolist())))
        z = flatten_list(z)
        return z
    except:
        return a

sd = locator(cities['City'])   
cities['State'] = locator(cities['City'])
cities

,City,State
0,Pune,Maharashtra
1,Mumbai,Maharashtra
2,Mumbai,Maharashtra
3,Pune,Maharashtra
